In [7]:
# Dependencies
import os
import pathlib
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from uszipcode import SearchEngine

# Import the API key
from config import geoapify_key
from config import census_key

DATA CLEANUP

1. The first set of data was obtained from the OpenBreweryAPI. The API is a free API, however it limits the number of queries to 200 rows and restricting sourcing of data to 2 pages at a time. In order to source all necessary data, it was necessary to create a loop to move through all 156 pages of the API and collect 100 cases per page or 200 every two pages. 

In [8]:
# URL for GET requests to retrieve brewery data
base_url = 'https://api.openbrewerydb.org/v1/breweries?by_country=United_States'

# Define an empty list to fetch the page data for each set of data
brewery_data = []

# Loop through the pages to get all the data
for page in range(1, 156):
    query_url = base_url + "&page="+ str(page) + "&per_page=200"
    response = requests.get(query_url).json()
    brewery_data.append(response)

# Create a list of all the breweries
breweries = []
for page in brewery_data:
    for brewery in page:
        breweries.append(brewery)

# Create a dataframe from the list of breweries
breweries_df = pd.DataFrame(breweries)

breweries_df.head()

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,None,Norman,Oklahoma,73069-8224,United States,-97.46818222,35.25738891,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,None,None,Austin,Texas,78745-1197,United States,None,None,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200
2,ef970757-fe42-416f-931d-722451f1f59c,10 Barrel Brewing Co,large,1501 E St,None,None,San Diego,California,92101-6618,United States,-117.129593,32.714813,6195782311,http://10barrel.com,California,1501 E St
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,None,None,Bend,Oregon,97701-9847,United States,-121.281706,44.08683531,5415851007,http://www.10barrel.com,Oregon,62970 18th St
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,None,None,Bend,Oregon,97703-2465,United States,-121.3288021,44.0575649,5415851007,None,Oregon,1135 NW Galveston Ave Ste B


The describe function below shows that 2,392 rows are misisng the longitude and latitude. Since the number of rows consitutde 30% of the full data set, 
it was important to backfill the missing information.   

In [ ]:
#creates a summary table of the data
breweries_df.describe()

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
count,7936,7936,7936,7153,4,0,7936,7936,7936,7936,5544,5544,7141,6787,7936,7153
unique,7936,7791,10,7064,4,0,2908,54,7695,2,5433,5433,6950,6437,54,7064
top,5128df48-79fc-4f0f-8b52-d06be54d0cec,Granite City Food & Brewery,micro,303 Main St,Estacada,NaN,Portland,California,64108,United States,-112.0773456,33.4485866,5122442739,http://www.gcfb.net,California,303 Main St
freq,1,6,4115,3,1,NaN,105,912,5,7935,6,6,5,23,912,3


In [10]:
#Prepare the breweries Database for analysis 
#drop brevery id, address_2, address_3, country, phone, website_url, and street
clean_breweries_df = breweries_df.drop(columns=['id', 'address_2', 'address_3', 'country', 'phone', 'website_url', 'street'])
clean_breweries_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.46818222,35.25738891,Oklahoma
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.08683531,Oregon
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.3288021,44.0575649,Oregon


Here's further confirmation that the necessary longtidue and latitude info is misisng in large numbers.   

In [11]:
#count missing values for each column
clean_breweries_df.isnull()
#count missing values for each column 
clean_breweries_df.isnull().sum()

name                 0
brewery_type         0
address_1          783
city                 0
state_province       0
postal_code          0
longitude         2392
latitude          2392
state                0
dtype: int64

Decided to use Geopify to pull latitude and longtiude for the missing zip codes. Since Geopify did not recognize the postal code +4 format; 
opted to use the uszipcode package to extract the first 5 digits. While long/lat can be also pulled via uszipcode package, the exercise was to demonstrate the use of API for sourcing additional information. Using the package will also allow pulling additional information later.  

In [12]:
#create a new column in dataframe with first 5 digits of zip code ensuring the information is stored as stings & recognized as zipcodes.
search = SearchEngine()

def extract_zipcode(zipcode):
    # Remove +4 extension if present
    zipcode = zipcode.split('-')[0]
    zipcode_obj = search.by_zipcode(zipcode)
    if zipcode_obj is not None:
        return str(zipcode_obj.zipcode)[:5].zfill(5)
    else:
        return None

clean_breweries_df['zip_code'] = clean_breweries_df['postal_code'].apply(extract_zipcode).astype(str)

#add a ditinct ID number to each row 
clean_breweries_df['brewery_id'] = clean_breweries_df.index + 1
clean_breweries_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.46818222,35.25738891,Oklahoma,73069,1
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas,78745,2
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.08683531,Oregon,97701,4
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.3288021,44.0575649,Oregon,97703,5


In [13]:
#load dataframe into csv
clean_breweries_df.to_csv('csv building blocks/breweries.csv', index=False)
#show count by the length of the postal code to see if there are any invalid postal codes
clean_breweries_df['zip_code'].str.len().value_counts()

5    7929
4       7
Name: zip_code, dtype: int64

In [14]:
#use dataframe from the csv breweries.csv
input_path = os.path.join("csv building blocks/breweries.csv")
csv_breweries_df = pd.read_csv(input_path)

In [15]:
# Can we delete this?

#Quick test if the API request url worked for a specific zip code
#postcode = "78745"
#target_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={postcode}&limit=1&type=postcode&format=json&apiKey={geoapify_key}"
#response = requests.get(target_url).json()
#print(json.dumps(response, indent=4, sort_keys=True))


Creating a dataframe with missing information to temporarily separate from the clean of the data.

In [16]:
#extract rows with missing longitude and create a new dataframe - all longitude missing values correspond to missinging latitude values
missing_long_df = csv_breweries_df[csv_breweries_df['longitude'].isnull()]

#create a new dataframe with only the missing longitude rowsbased on clean_breweries_df
missing_long_df = clean_breweries_df[clean_breweries_df['longitude'].isnull()]

missing_long_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas,78745,2
18,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,None,None,New York,14221,19
19,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,None,None,Arizona,85296,20
24,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,None,None,Minnesota,55101,25
26,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,None,None,South Carolina,29687,27


In [17]:
#drop the missing_long dataframe into a csv file
missing_long_df.to_csv('csv building blocks/breweries_missing_values.csv', index=False)

In [18]:
#use dataframe from the csv_breweries_missing_values.csv
input_path2 = os.path.join("csv building blocks/breweries_missing_values.csv")
csv_breweries_missing_df = pd.read_csv(input_path2)

Finally demonstrate use of Geopify for obtianing missing information - encountered errors were bypassed with the try/except. 

In [19]:
#for missing longitude, use geopify to look up the longitude and latitude
for index, row in csv_breweries_missing_df.iterrows():
    postcode = row['zip_code']
    target_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={postcode}&limit=1&type=postcode&format=json&apiKey={geoapify_key}"
    response = requests.get(target_url)

    try:
        json_response = json.loads(response.text)
        if json_response:
            csv_breweries_missing_df.loc[index, 'longitude'] = json_response['results'][0]['lon']
            csv_breweries_missing_df.loc[index, 'latitude'] = json_response['results'][0]['lat']
            print(f"index: {index}, postcode: {postcode}, longitude: {json_response['results'][0]['lon']}, latitude: {json_response['results'][0]['lat']}") 

        else:
            print(f"Error: Empty JSON response for index {index}, postcode {postcode}")
    except IndexError:
        print(f"Error: IndexError occurred for index {index}, postcode {postcode}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON response for index {index}, postcode {postcode}")

csv_breweries_missing_df.head()

index: 0, postcode: 78745, longitude: -97.792614846, latitude: 30.208605656
index: 1, postcode: 14221, longitude: -78.729920877, latitude: 42.980952841
index: 2, postcode: 85296, longitude: -111.762518433, latitude: 33.335136701
index: 3, postcode: 55101, longitude: -93.088300242, latitude: 44.955919207
index: 4, postcode: 29687, longitude: -82.327800669, latitude: 34.991467109
index: 5, postcode: 27603, longitude: -78.66089377, latitude: 35.71301003
index: 6, postcode: 56442, longitude: -94.116315338, latitude: 46.677860236
index: 7, postcode: 05478, longitude: -73.104390267, latitude: 44.803484605
index: 8, postcode: 21157, longitude: -76.983937143, latitude: 39.558197671
index: 9, postcode: 22553, longitude: -77.60125509, latitude: 38.231176411
index: 10, postcode: 11741, longitude: -73.070358674, latitude: 40.794962148
index: 11, postcode: 05452, longitude: -73.08637315, latitude: 44.51595767
index: 12, postcode: 20109, longitude: -77.506105128, latitude: 38.784153209
index: 13, po

KeyboardInterrupt: 

In [ ]:
#save the dataframe with filled info into a csv file csv_breweries_missing_completed_df
csv_breweries_missing_df.to_csv('csv building blocks/breweries_missing_completed.csv', index=False)
csv_breweries_missing_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,-97.792615,30.208606,Texas,78745,2
1,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,-78.729921,42.980953,New York,14221,19
2,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,-111.762518,33.335137,Arizona,85296,20
3,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,-93.088300,44.955919,Minnesota,55101,25
4,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,-82.327801,34.991467,South Carolina,29687,27


In [ ]:
# Drop rows with blank long/lat values from total set of breweries 
breweries_dropped_nan = csv_breweries_df.dropna(subset=["longitude", "latitude"])

# Output list of breweries with no long/lat blanks to csv to combine with missing data 
breweries_dropped_nan.to_csv("csv building blocks/breweries_dropped.csv", index=False)
breweries_dropped_nan.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,1
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,4
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,5
5,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,6


In [20]:
# Read in csv files to combine 
df1 = pd.read_csv("csv building blocks/breweries_dropped.csv")
df2 = pd.read_csv("csv building blocks/breweries_missing_completed.csv")

# Combine csv files 
df_combined = pd.concat([df1, df2])

# Four rows with blank long/lat values still appearing; drop those rows 
complete_breweries = df_combined.dropna(subset=["longitude", "latitude"])

# Output list of breweries with complete long/lat values 
complete_breweries.to_csv("csv building blocks/breweries_complete.csv", index=False)
complete_breweries.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,1
1,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
2,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,4
3,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,5
4,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,6


In [22]:
# Loop through rows of completed brewery data to get final row count to confirm succsefull combination 
rowcount  = 0
#iterating through the whole file
for row in open("csv building blocks/breweries_complete.csv"):
  rowcount+= 1
 #printing the result
print("Number of rows:-", rowcount)

Number of rows:- 7933


In [23]:
# Base URL for census
base_url = 'https://api.census.gov/data/2021/acs/acs1/profile?'

# Define the parameters
# Parameters google sheet link https://docs.google.com/spreadsheets/d/1JolzmtPdrUa3RR0SHg5CqdUon5I4xVMXJgzAcMjCryk/edit#gid=0
parameter_list = ['DP02_0001E','DP02_0006E','DP02_0010E','DP02_0025E','DP02_0025PE','DP02_0031E','DP02_0031PE','DP02_0067E','DP02_0067PE','DP02_0068E',
'DP02_0068PE','DP03_0001PE','DP03_0001E','DP03_0051E','DP03_0062E','DP03_0063E','DP03_0075E','DP03_0086E','DP03_0087E','DP03_0088E','DP03_0089E'
,'DP03_0090E','DP03_0091E','DP03_0119E','DP03_0119PE','DP03_0128E','DP03_0128PE','DP04_0001E','DP04_0001PE','DP04_0006E','DP04_0006PE','DP05_0001E',
'DP05_0022E','DP05_0022PE','DP05_0008E','DP05_0008PE','DP05_0026E','DP05_0026PE','DP05_0027E','DP05_0027PE','DP03_0089E']

# Define an empty list to fetch the census data for each parameter
census_data = []

# Loop through parameter list
for parameter in parameter_list:
    query_url = base_url + "get=NAME," + parameter + "&for=place:*&in=state:*" + "&key=" + census_key 
    response = requests.get(query_url).json()
    census_data.append(response)

# Index the first element in census data
df_data = census_data[0]

# Index the first element in df_data, this is the column headers
cols = df_data[0]

# Create a dictionary of the data for the dataframe
rows = [dict(zip(cols, row)) for row in df_data[1:]]

# Create the dataframe 
df = pd.DataFrame(rows)

# Create the column names
df = df[['NAME','state','place',cols[1]]]

# Loop through the census data to add all the parameters
for d in census_data:

    df[d[0][1]] = [row[1] for row in d[1:]]

df.to_csv("csv building blocks/census_codes.csv", index=False)

census_final = df.rename(columns={'DP02_0001E':'households_total',
'DP02_0006E':'households_total_male',
'DP02_0010E':'households_total_female',
'DP02_0025E':'marital_status__male_15_over',
'DP02_0025PE':'marital_status__male_15_over_percent',
'DP02_0031E':'marital_status__female_15_over',
'DP02_0031PE':'marital_status__female_15_over_percent',
'DP02_0067E':'edu_high_school_over',
'DP02_0067PE':'edu_high_school_over',
'DP02_0068E':'edu_bach_degree_over',
'DP02_0068PE':'edu_bach_degree_over_percent',
'DP03_0001PE':'employment_status_percent',
'DP03_0001E':'employment_status',
'DP03_0051E':'income_household',
'DP03_0062E':'income_household_median',
'DP03_0063E':'income_household_mean',
'DP03_0075E':'income_family',
'DP03_0086E':'income_family_median',
'DP03_0087E':'income_family_mean',
'DP03_0088E':'income_per_capita',
'DP03_0089E':'income_nonfamily',
'DP03_0090E':'income_nonfamily_median',
'DP03_0091E':'income_nonfamily_mean',
'DP03_0119E':'income_below_poverty_family',
'DP03_0119PE':'income_below_poverty_percent_family',
'DP03_0128E':'income_below_poverty',
'DP03_0128PE':'income_below_poverty_percent',
'DP04_0001E':'housing_occupancy',
'DP04_0001PE':'housing_occupancy_percent',
'DP04_0006E':'housing_in_structure',
'DP04_0006PE':'housing_in_structure_percent',
'DP05_0001E':'population_total',
'DP05_0022E':'population_over_21',
'DP05_0022PE':'population_over_21_percent',
'DP05_0008E':'population_15_to_19',
'DP05_0008PE':'population_15_to_19_percent',
'DP05_0026E':'population_male_over_18',
'DP05_0026PE':'population_male_over_18_percent',
'DP05_0027E':'population_female_over_18',
'DP05_0027PE':'population_female_over_18_percent'})

census_final.head()

,NAME,state,place,households_total,households_total_male,households_total_female,marital_status__male_15_over,marital_status__male_15_over_percent,marital_status__female_15_over,marital_status__female_15_over_percent,...,housing_in_structure_percent,population_total,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent
0,"O'Fallon city, Missouri",29,54074,34412,4333,7513,34707,34707,39512,39512,...,35283,93651,66311,70.8,7222,7.7,32890,47.2,36774,52.8
1,"St. Louis city, Missouri",29,65000,139736,37975,57478,118761,118761,128187,128187,...,173493,293310,228212,77.8,14865,5.1,114730,47.9,124557,52.1
2,"Passaic city, New Jersey",34,56550,20446,2636,7469,24484,24484,27650,27650,...,21049,69637,44179,63.4,6414,9.2,22608,46.9,25559,53.1
3,"Nashua city, New Hampshire",33,50260,36986,8301,8689,38993,38993,37580,37580,...,38955,91122,70673,77.6,5215,5.7,37699,51.2,35930,48.8
4,"Rochester city, Minnesota",27,54880,49984,8903,13809,46609,46609,51593,51593,...,54111,121471,88939,73.2,8678,7.1,44572,47.7,48846,52.3


In [24]:
# Create csv files of census with updated column names
census_final.to_csv("csv building blocks/census_names.csv", index=False)

In [25]:
# Add index label to final census
census_final.to_csv("csv building blocks/final_census_nl.csv", index_label="City_ID")

In [26]:
# 

# Study data files
#brewery_last_path = "csv building blocks/breweries_missing_completed.csv"
#census_last_path = "csv building blocks/final_census_nl.csv"

# Read the mouse data and the study results
#brewery_last = pd.read_csv(brewery_last_path)
#census_last = pd.read_csv(census_last_path, index_col="City_ID")

In [27]:
breweries_completed = "csv building blocks/breweries_complete.csv"
census_completed = "csv building blocks/final_census_nl.csv"

# Read the mouse data and the study results
brewery_last = pd.read_csv(breweries_completed)
census_last = pd.read_csv(census_completed, index_col="City_ID")

In [28]:
# Display dataframe 1
#rename city column to City
brewery_last = brewery_last.rename(columns={"city":"City","state_province":"State"})
brewery_last.head()

#brewery_last[brewery_last["name"] == 'Von Ebert Brewing']

,name,brewery_type,address_1,City,State,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,1
1,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
2,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,4
3,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,5
4,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,6


In [29]:
# Display dataframe 2
census_last.head()

,NAME,state,place,households_total,households_total_male,households_total_female,marital_status__male_15_over,marital_status__male_15_over_percent,marital_status__female_15_over,marital_status__female_15_over_percent,...,housing_in_structure_percent,population_total,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent
City_ID,,,,,,,,,,,,,,,,,,,,,
0,"O'Fallon city, Missouri",29,54074,34412.0,4333.0,7513.0,34707.0,34707.0,39512.0,39512.0,...,35283,93651,66311,70.8,7222,7.7,32890,47.2,36774,52.8
1,"St. Louis city, Missouri",29,65000,139736.0,37975.0,57478.0,118761.0,118761.0,128187.0,128187.0,...,173493,293310,228212,77.8,14865,5.1,114730,47.9,124557,52.1
2,"Passaic city, New Jersey",34,56550,20446.0,2636.0,7469.0,24484.0,24484.0,27650.0,27650.0,...,21049,69637,44179,63.4,6414,9.2,22608,46.9,25559,53.1
3,"Nashua city, New Hampshire",33,50260,36986.0,8301.0,8689.0,38993.0,38993.0,37580.0,37580.0,...,38955,91122,70673,77.6,5215,5.7,37699,51.2,35930,48.8
4,"Rochester city, Minnesota",27,54880,49984.0,8903.0,13809.0,46609.0,46609.0,51593.0,51593.0,...,54111,121471,88939,73.2,8678,7.1,44572,47.7,48846,52.3


In [30]:
# split NAME into two columns where (,)
new = census_last["NAME"].str.split(",", n = 1, expand = True)
#rename column 0 to "City"
new.rename(columns={0:'City'}, inplace=True)
#rename column 1 to "State"
new.rename(columns={1:'State'}, inplace=True)
#replace column with the new columns
census_last['City'] = new['City']
census_last['State'] = new['State']
#delete space after the City name in each row
census_last['City'] = census_last['City'].str[:-1]
#delete space ahead of the State name in each row
census_last['State'] = census_last['State'].str[1:]
#drop column NAME
#census_last.drop(columns =["NAME"], inplace = True)
#drop last 4 characters and leading space form the"City" column
census_last['City'] = census_last['City'].str[:-4]
census_last.head()

,NAME,state,place,households_total,households_total_male,households_total_female,marital_status__male_15_over,marital_status__male_15_over_percent,marital_status__female_15_over,marital_status__female_15_over_percent,...,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent,City,State
City_ID,,,,,,,,,,,,,,,,,,,,,
0,"O'Fallon city, Missouri",29,54074,34412.0,4333.0,7513.0,34707.0,34707.0,39512.0,39512.0,...,66311,70.8,7222,7.7,32890,47.2,36774,52.8,O'Fallon,Missouri
1,"St. Louis city, Missouri",29,65000,139736.0,37975.0,57478.0,118761.0,118761.0,128187.0,128187.0,...,228212,77.8,14865,5.1,114730,47.9,124557,52.1,St. Louis,Missouri
2,"Passaic city, New Jersey",34,56550,20446.0,2636.0,7469.0,24484.0,24484.0,27650.0,27650.0,...,44179,63.4,6414,9.2,22608,46.9,25559,53.1,Passaic,New Jersey
3,"Nashua city, New Hampshire",33,50260,36986.0,8301.0,8689.0,38993.0,38993.0,37580.0,37580.0,...,70673,77.6,5215,5.7,37699,51.2,35930,48.8,Nashua,New Hampshire
4,"Rochester city, Minnesota",27,54880,49984.0,8903.0,13809.0,46609.0,46609.0,51593.0,51593.0,...,88939,73.2,8678,7.1,44572,47.7,48846,52.3,Rochester,Minnesota


In [31]:
#census_last[census_last["City"] == 'Portland']
census_last.to_csv("csv building blocks/census_last.csv", index=False)


In [32]:
#Weeding
#replace Anchorage municip with Anchorage
census_last['City'] = census_last['City'].replace({'Anchorage Municipality':'Anchorage'})
#replace "salt lake city" with "Salt Lake City" in breweries
brewery_last['City'] = brewery_last['City'].replace({'salt lake city':'Salt Lake City'})
#replace San Buenaventura (Ventura) with Ventura in census
census_last['City'] = census_last['City'].replace({'San Buenaventura (Ventura)':'Ventura'})
#replace "St. Petersburg" with "St Petersburg" in census
census_last['City'] = census_last['City'].replace({'St. Petersburg':'St Petersburg'})
#replace "St. Louis" with "St Louis" in census
census_last['City'] = census_last['City'].replace({'Athens-Clarke County unified government (balance)':'Athens'})
#replace Urban Honolul with Honolulu in census
census_last['City'] = census_last['City'].replace({'Urban Honolulu':'Honolulu'})
#reaplce Indianapolis city (bal with Indianapolis in census
census_last['City'] = census_last['City'].replace({'Indianapolis city (balance)':'Indianapolis'})
#replace Louisville/Jefferson County metro government (balance) with Louisville in census
census_last['City'] = census_last['City'].replace({'Louisville/Jefferson County metro government (balance)':'Louisville'})
#replace Lexington-Fayette urban county with Lexington in census
census_last['City'] = census_last['City'].replace({'Lexington-Fayette urban county':'Lexington'})
#replace Nashville-Davidson metropolitan government (balance) with Nashville in census
census_last['City'] = census_last['City'].replace({'Nashville-Davidson metropolitan government (balance)':'Nashville'})



In [33]:
#merge two dataframes by City column 
merged_df = pd.merge(brewery_last, census_last, on=["City","State"], how="left")

#drop merged df into csv file
merged_df.to_csv("csv building blocks/merged_df.csv", index=False)
merged_df.head()

#merged_df[merged_df["name"] == 'Von Ebert Brewing']
#brewery_last[brewery_last["name"] == 'Von Ebert Brewing']

,name,brewery_type,address_1,City,State,postal_code,longitude,latitude,state_x,zip_code,...,housing_in_structure_percent,population_total,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,...,57307.0,128087.0,92914.0,72.5,12803.0,10.0,52250.0,48.7,54931.0,51.3
1,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,...,555456.0,1381600.0,1057949.0,76.6,87255.0,6.3,574112.0,51.2,547406.0,48.8
2,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,...,47882.0,102079.0,80025.0,78.4,5002.0,4.9,40641.0,49.5,41473.0,50.5
3,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,...,47882.0,102079.0,80025.0,78.4,5002.0,4.9,40641.0,49.5,41473.0,50.5
4,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,...,303483.0,642218.0,515575.0,80.3,31016.0,4.8,264448.0,49.4,270792.0,50.6


In [34]:
# Remove cities without census information
dfresult = merged_df.dropna(thresh = 43)

# Check the row count of the final dataset
len(dfresult.index)


3168

In [35]:
# Create final merged dataframe
dfresult.to_csv("csv building blocks/merged_df_cleaned.csv", index=False)

In [36]:
# Can we delete this??

# create a summary dataframe with number of breweries by zip_code - listing City, State, and Zip Code, and brewery count  

# load additional information regrdidng the zipcode/each brewery cluster data from https://uszipcode.readthedocs.io/
# review documentation by following the link to see how the information populates and if useful for our analysis
# Some of the data to consider --> household_income; median_household_income population, population density, families_vs_singles, 
# median_home_value, educational_attainment_for_population_25_and_over 




In [37]:
# Can we delete this??

#explore Census API Community Data API for any additional information that might be useful for our analysis - gender, race etc.
#explore articles for inspiration on what other data might be useful for our analysis
# https://www.census.gov/library/visualizations/interactive/breweries.html  --> Article about breweries for the Census 
#https://cbb.census.gov/cbb/#industry0=312120&geoId=17031&geoType=county&view=report&reportType=summary --> Brewery Busisness in Illinois 


#load relevant data into the zipcode summary table 